In [3]:
#import libraries
import pandas as pd
from tqdm import tqdm
import requests
from tqdm import tqdm
from concurrent.futures import as_completed, ThreadPoolExecutor
from datetime import datetime, timedelta
import time
import json
import calendar

#read csv file and initialize api key
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/out.csv')
API_KEY = '8GPFAUV4AAH3IANWKTV2SI93R9J5PMBXXJ'

#drop the "Unnamed" column
df.drop(df.filter(regex='Unnamed'),axis=1, inplace=True)

df

,name,contract_address
0,Origin Protocol,0x8207c1ffc5b6804f6024322ccf34f29c3541ae26
1,CWallet,0xd55236d48606c295adebf129dad04fc74bfaa708
2,Ethereum Push Notification Service,0xf418588522d5dd018b425E472991E52EBBeEEEEE
3,SKALE Network,0x00c83aecc790e8a4453e5dd3b0b4b3680501a7a7
4,Magic Internet Money,0x99d8a9c45b2eca8864373a26d1459e3dff1e17f3
...,...,...
141,DeversiFi,0xdddddd4301a082e62e84e43f474f044423921918
142,Degenerator Meme,0xd5525d397898e5502075ea5e830d8914f6f0affe
143,Volatility Protocol Token,0x5166E09628b696285E3A151e84FB977736a83575
144,Portion,0x6D0F5149c502faf215C89ab306ec3E50b15e2892


#24h transactions

In [4]:
txlist24 = []
volume7d = []
volume = 0
data = []

#Get the current unix time in UTC
today = datetime.utcnow()
current_unix_time = calendar.timegm(today.utctimetuple())

ONE_DAY = 86400
ONE_WEEK = 7*86400

#Go through each contract address
for i in tqdm(df['contract_address']):
  #Use Etherscan API to get a list of 10000 transactions from the address
  url = f'http://api.etherscan.io/api?module=account&action=txlist&address={i}&startblock=0&endblock=latest&sort=desc&offset=10000&page=0&apikey={API_KEY}'
  response = requests.get(url).json()
  #Find every transaction that is within 24h/7d
  for x in response['result']:
    #If within 24 hours, add it to the transaction list
    if current_unix_time - int(x['timeStamp']) <= ONE_DAY:
      txlist24.append(x)
    #If within 7 days, update the 7dvolume counter
    if current_unix_time - int(x['timeStamp']) <= ONE_WEEK:
      volume+=1
  volume7d.append(volume)
  volume = 0
  data.append(txlist24)
  txlist24 = []

#Update the dataframe with the transaction values
df['24htransactions'] = data
df

100%|██████████| 146/146 [07:46<00:00,  3.19s/it]


,name,contract_address,24htransactions
0,Origin Protocol,0x8207c1ffc5b6804f6024322ccf34f29c3541ae26,"[{'blockNumber': '15034766', 'timeStamp': '165..."
1,CWallet,0xd55236d48606c295adebf129dad04fc74bfaa708,"[{'blockNumber': '15033719', 'timeStamp': '165..."
2,Ethereum Push Notification Service,0xf418588522d5dd018b425E472991E52EBBeEEEEE,"[{'blockNumber': '15034551', 'timeStamp': '165..."
3,SKALE Network,0x00c83aecc790e8a4453e5dd3b0b4b3680501a7a7,"[{'blockNumber': '15034794', 'timeStamp': '165..."
4,Magic Internet Money,0x99d8a9c45b2eca8864373a26d1459e3dff1e17f3,"[{'blockNumber': '15034606', 'timeStamp': '165..."
...,...,...,...
141,DeversiFi,0xdddddd4301a082e62e84e43f474f044423921918,[]
142,Degenerator Meme,0xd5525d397898e5502075ea5e830d8914f6f0affe,"[{'blockNumber': '15033642', 'timeStamp': '165..."
143,Volatility Protocol Token,0x5166E09628b696285E3A151e84FB977736a83575,[]
144,Portion,0x6D0F5149c502faf215C89ab306ec3E50b15e2892,"[{'blockNumber': '15032764', 'timeStamp': '165..."


#24h/7d volume

In [5]:
#Get the 24 hour volume and add it to the dataframe
volume24h = []
for i in data:
  volume24h.append(len(i))

df.insert(1, '24hvolume', volume24h)

#Get the 7 day volume and add it to the dataframe
df.insert(1, '7dvolume', volume7d)
df

,name,7dvolume,24hvolume,contract_address,24htransactions
0,Origin Protocol,1175,220,0x8207c1ffc5b6804f6024322ccf34f29c3541ae26,"[{'blockNumber': '15034766', 'timeStamp': '165..."
1,CWallet,7,1,0xd55236d48606c295adebf129dad04fc74bfaa708,"[{'blockNumber': '15033719', 'timeStamp': '165..."
2,Ethereum Push Notification Service,112,18,0xf418588522d5dd018b425E472991E52EBBeEEEEE,"[{'blockNumber': '15034551', 'timeStamp': '165..."
3,SKALE Network,404,54,0x00c83aecc790e8a4453e5dd3b0b4b3680501a7a7,"[{'blockNumber': '15034794', 'timeStamp': '165..."
4,Magic Internet Money,230,24,0x99d8a9c45b2eca8864373a26d1459e3dff1e17f3,"[{'blockNumber': '15034606', 'timeStamp': '165..."
...,...,...,...,...,...
141,DeversiFi,2,0,0xdddddd4301a082e62e84e43f474f044423921918,[]
142,Degenerator Meme,15,3,0xd5525d397898e5502075ea5e830d8914f6f0affe,"[{'blockNumber': '15033642', 'timeStamp': '165..."
143,Volatility Protocol Token,1,0,0x5166E09628b696285E3A151e84FB977736a83575,[]
144,Portion,65,2,0x6D0F5149c502faf215C89ab306ec3E50b15e2892,"[{'blockNumber': '15032764', 'timeStamp': '165..."


#Transaction Hashes


In [6]:
#Get a list of transaction hashes for the transactions in 24h

txhash = []
txhash_list = []
for i in df['24htransactions']:
  for j in i:
    txhash.append(j['hash'])
  txhash_list.append(txhash)
  txhash = []

df.insert(1, '24htransaction_hashes', txhash_list)
df

,name,24htransaction_hashes,7dvolume,24hvolume,contract_address,24htransactions
0,Origin Protocol,[0x53c84d91a31442c6a55ea689ea4b58a21704fdfcc3e...,1175,220,0x8207c1ffc5b6804f6024322ccf34f29c3541ae26,"[{'blockNumber': '15034766', 'timeStamp': '165..."
1,CWallet,[0xd7c0f992439e78b7067426cfeef0ac317a65fccf98f...,7,1,0xd55236d48606c295adebf129dad04fc74bfaa708,"[{'blockNumber': '15033719', 'timeStamp': '165..."
2,Ethereum Push Notification Service,[0x4bc222b5683fb42c88c15a2e069efe9a681adb7ab59...,112,18,0xf418588522d5dd018b425E472991E52EBBeEEEEE,"[{'blockNumber': '15034551', 'timeStamp': '165..."
3,SKALE Network,[0xac80c92619a609f5a982911d24092729ae7d1a202d3...,404,54,0x00c83aecc790e8a4453e5dd3b0b4b3680501a7a7,"[{'blockNumber': '15034794', 'timeStamp': '165..."
4,Magic Internet Money,[0xbfecc2fbf13df7e9eac8e2deb41e5bdd2d2ddfa0142...,230,24,0x99d8a9c45b2eca8864373a26d1459e3dff1e17f3,"[{'blockNumber': '15034606', 'timeStamp': '165..."
...,...,...,...,...,...,...
141,DeversiFi,[],2,0,0xdddddd4301a082e62e84e43f474f044423921918,[]
142,Degenerator Meme,[0x4523e24a992bd60746194c28ef236a14a30f48cb039...,15,3,0xd5525d397898e5502075ea5e830d8914f6f0affe,"[{'blockNumber': '15033642', 'timeStamp': '165..."
143,Volatility Protocol Token,[],1,0,0x5166E09628b696285E3A151e84FB977736a83575,[]
144,Portion,[0x6e8ec3af5691034177b8d175968d6325c0b0c566884...,65,2,0x6D0F5149c502faf215C89ab306ec3E50b15e2892,"[{'blockNumber': '15032764', 'timeStamp': '165..."


#Output

In [7]:
#Remove contract_addresses and output to csv
df.drop('contract_address', axis=1, inplace=True)
df.to_csv('finalOutput.csv')